Ultimate guitar converter
==============

convert any song on https://www.ultimate-guitar.com/ to a LaTeX file.

In [6]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [36]:
#==============================================================================
# scrape the content from https://www.ultimate-guitar.com/
#==============================================================================

# modules used to get and parse the content
    import requests
from bs4 import BeautifulSoup

def scrape(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    # the main part of the song can be found under
    txt = soup.find_all('textarea')[1].get_text()

    info = soup.find('title').get_text().title()
    title = info[:info.find("By")-1]
    interpret = info[info.find("By")+3:info.find("@")-1]
    
    return (txt,title,interpret)


In [30]:
#==============================================================================
# convert the raw input to a LaTeX format
#==============================================================================

numbers = ['1','2','3','4','5','6','7','8','9',' ']

# function to split line into single chords
def split_row(row):
    line = []
    while row:
        try: item, row = row.lstrip().split(' ', 1)
        except: item, row = row.lstrip(), ''
        line.append(item)
    return line

# *************************************
# format for LaTeX
# *************************************
def convert_song(txt):
    #split input into list
    INPUT = txt.split('\n')
    
    
    length = len(INPUT)
    INPUT.append('')
    OUTPUT = []
    endenv = ''
    i = 0

    OUTPUT.append('\\begin{song}[title={' + titlefield.value + '},interpret={' + interpretfield.value +'}]')
    # main loop
    while i < length:
        row = INPUT[i]
        
        # use empty space p to determine if a row is a "chord-row"
        if row != '':
            p = (row.count(' ') / len(row))
        else:
            p = 0
            
        if row == '':
            if endenv != '':
                OUTPUT.append(endenv)
                OUTPUT.append('')
                endenv = ''
            i += 1
        # determine environment
        elif row[0] == '[':
            env = row[row.find("[")+1:row.find("]")].lower()
            for string in numbers: env = env.replace(string,'')
            OUTPUT.append('\\begin{' + env + '}')
            endenv = '\\end{' + env + '}'
            i += 1
            if INPUT[i] == '': i += 1
     
        # place chords over text
        elif p >.50:
            # determine chord and position
            chords = split_row(row)
            index = []
            j = 0
            for chord in chords:
                idx = row.find(chord,j)
                index.append(idx)
                j = idx

            # create new line for text + chords
            textrow = INPUT[i+1]

            # check if textrow is long enough
            if len(textrow)<index[-1]:
                textrow += ' '* (index[-1]-len(textrow)+1)

            for chord, idx in zip(reversed(chords), reversed(index)):
                textrow = textrow[:idx] + '\chord{' + chord + '}' + textrow[idx:]
            textrow += '\\\\'

            # append result
            OUTPUT.append(textrow.replace('”','"').replace('“','"').replace('#','\sharp').replace("’","'"))
            i += 2 
        else:
            OUTPUT.append(row + '\\\\' )
            i += 1
        
    OUTPUT.append('\\end{song}')
    
    txt = ''
    for line in OUTPUT:
        txt += line + '\n'
        
    return txt


In [40]:
import io

def save(txt):
    #*************************************
    # write to file with utf8 encoding
    #*************************************
        
    filename = titlefield.value.lower() + '.tex'
    with io.open(filename,'w',encoding='utf8') as f:
        f.write(txt)
    f.close()

In [38]:
from ipywidgets import widgets,HBox, VBox, Layout
from IPython.display import display

button1 = widgets.Button(description="get song",layout=Layout(height='30px', width='200px'))
button2 = widgets.Button(description="convert",layout=Layout(height='30px', width='300px'))
button3 = widgets.Button(description="save",layout=Layout(height='30px', width='300px'))

txtfield = widgets.Textarea(placeholder='some songtext',layout=Layout(height='600px', width='600px'))
urlfield = widgets.Text(placeholder='type in URL',layout=Layout(height='30px', width='400px'))
titlefield = widgets.Text(placeholder='title',layout=Layout(height='30px', width='300px'))
interpretfield = widgets.Text(placeholder='interpret',layout=Layout(height='30px', width='300px'))

def on_button1_clicked(b):
    if urlfield.value != '':
        txt,title,interpret = scrape(urlfield.value)
        txtfield.value = txt
        titlefield.value = title
        interpretfield.value = interpret
    else:
        txtfield.placeholder = 'invalid url'
    
def on_button2_clicked(b):
    txtfield.value = convert_song(txtfield.value)
    
def on_button3_clicked(b):
    save(txtfield.value)    
    
button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)
button3.on_click(on_button3_clicked)

# Layout of the GUI
top_box = HBox([urlfield,button1])
mid_box = HBox([titlefield,interpretfield])
bottom_box = HBox([button2,button3])

VBox([top_box,mid_box,txtfield,bottom_box])